<img src="images/deeplearningwizard_web_logo.png" alt="deeplearningwizard" style="width: 100px;"/>

# 3. Learning Rate Scheduling

## Optimization Algorithm 3: Mini-batch Gradient Descent (SGD)
- Combination of batch gradient descent & stochastic gradient descent
    - $\theta = \theta - \eta \cdot  \nabla J(\theta, x^{i: i+n}, y^{i:i+n})$
- Characteristics
    - Compute the gradient of the lost function w.r.t. parameters for **n sets of training sample (n input and n label)**, $\nabla J(\theta, x^{i: i+n}, y^{i:i+n})$
    - Use this to update our parameters at every iteration
    
## Learning Intuition Recap
- Learning process
    - Original parameters $\rightarrow$ given input, get output $\rightarrow$ compare with labels $\rightarrow$ get loss with comparison of input/output $\rightarrow$ get gradients of loss w.r.t parameters $\rightarrow$ **update parameters so model can churn output closer to labels** $\rightarrow$ repeat


## Learning Rate Pointers
![](./images/lr1.png)
- **Update parameters so model can churn output closer to labels, lower loss**
    - $\theta = \theta - \eta \cdot  \nabla J(\theta, x^{i: i+n}, y^{i:i+n})$
- If we set $\eta$ to be a **large value** $\rightarrow$ learn too much (rapid learning)
    - Unable to converge to a good local minima (unable to effectively gradually decrease your loss, overshoot the local lowest value)
- If we set $\eta$ to be a **small value** $\rightarrow$ learn too little (slow learning)
    - May take too long or unable to convert to a good local minima

## Need for Learning Rate Schedules

- Benefits
    - Converge faster
    - Higher accuracy
![](./images/lr2.png)

## Top Learning Rate Schedules

1. Step-wise Decay 
2. Reduce on Loss Plateau Decay

## 1. Step-wise Decay: Every Epoch
- At every epoch,
    - $ \eta_t = \eta_{t-1}\gamma $
    - $\gamma = 0.1$
- Optimization Algorithm 4: SGD Nesterov
    - Modification of SGD Momentum 
        - $v_t = \gamma v_{t-1} + \eta \cdot  \nabla J(\theta - \gamma v_{t-1}, x^{i: i+n}, y^{i:i+n})$
        - $\theta = \theta - v_t$
- Practical example
    - Given $ \eta_t = 0.1 $ and $ \gamma = 0.01$
    - Epoch 0: $ \eta_t = 0.1 $
    - Epoch 1: $ \eta_{t+1} = 0.1 (0.1) =  0.01$
    - Epoch 2: $ \eta_{t+2} = 0.1 (0.1)^2 =  0.001$
    - Epoch n: $ \eta_{t+n} = 0.1 (0.1)^n$

In [9]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import StepLR

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# step_size: at how many multiples of epoch you decay
# step_size = 1, after every 1 epoch, new_lr = lr*gamma 
# step_size = 2, after every 2 epoch, new_lr = lr*gamma 

# gamma = decaying factor
scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    # Decay Learning Rate
    scheduler.step()
    # Print Learning Rate
    print('Epoch:', epoch,'LR:', scheduler.get_lr())
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

Epoch: 0 LR: [0.1]
Iteration: 500. Loss: 0.25232651829719543. Accuracy: 95.66
Epoch: 1 LR: [0.010000000000000002]
Iteration: 1000. Loss: 0.07556439191102982. Accuracy: 97.1
Epoch: 2 LR: [0.0010000000000000002]
Iteration: 1500. Loss: 0.08089657127857208. Accuracy: 97.14
Epoch: 3 LR: [0.00010000000000000003]
Iteration: 2000. Loss: 0.03777354583144188. Accuracy: 97.16
Epoch: 4 LR: [1.0000000000000003e-05]
Iteration: 2500. Loss: 0.039101023226976395. Accuracy: 97.16
Iteration: 3000. Loss: 0.0747142806649208. Accuracy: 97.16


## 1. Step-wise Decay: Every 2 Epochs
- At every 2 epoch,
    - $ \eta_t = \eta_{t-1}\gamma $
    - $\gamma = 0.1$
- Optimization Algorithm 4: SGD Nesterov
    - Modification of SGD Momentum 
        - $v_t = \gamma v_{t-1} + \eta \cdot  \nabla J(\theta - \gamma v_{t-1}, x^{i: i+n}, y^{i:i+n})$
        - $\theta = \theta - v_t$
- Practical example
    - Given $ \eta_t = 0.1 $ and $ \gamma = 0.01$
    - Epoch 0: $ \eta_t = 0.1 $
    - Epoch 1: $ \eta_{t+1} = 0.1$
    - Epoch 2: $ \eta_{t+2} = 0.1 (0.1) =  0.01$

In [10]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import StepLR

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# step_size: at how many multiples of epoch you decay
# step_size = 1, after every 2 epoch, new_lr = lr*gamma 
# step_size = 2, after every 2 epoch, new_lr = lr*gamma 

# gamma = decaying factor
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    # Decay Learning Rate
    scheduler.step()
    # Print Learning Rate
    print('Epoch:', epoch,'LR:', scheduler.get_lr())
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

Epoch: 0 LR: [0.1]
Iteration: 500. Loss: 0.25232651829719543. Accuracy: 95.66
Epoch: 1 LR: [0.1]
Iteration: 1000. Loss: 0.07672126591205597. Accuracy: 96.47
Epoch: 2 LR: [0.010000000000000002]
Iteration: 1500. Loss: 0.07323123514652252. Accuracy: 97.5
Epoch: 3 LR: [0.010000000000000002]
Iteration: 2000. Loss: 0.045940183103084564. Accuracy: 97.62
Epoch: 4 LR: [0.0010000000000000002]
Iteration: 2500. Loss: 0.029797477647662163. Accuracy: 97.56
Iteration: 3000. Loss: 0.034976325929164886. Accuracy: 97.58


## 1. Step-wise Decay: Every Epoch, Larger Gamma
- At every epoch,
    - $ \eta_t = \eta_{t-1}\gamma $
    - $\gamma = 0.96$
- Optimization Algorithm 4: SGD Nesterov
    - Modification of SGD Momentum 
        - $v_t = \gamma v_{t-1} + \eta \cdot  \nabla J(\theta - \gamma v_{t-1}, x^{i: i+n}, y^{i:i+n})$
        - $\theta = \theta - v_t$
- Practical example
    - Given $ \eta_t = 0.1 $ and $ \gamma = 0.96$
    - Epoch 1: $ \eta_t = 0.1 $
    - Epoch 2: $ \eta_{t+1} = 0.1 (0.96) =  0.096$
    - Epoch 3: $ \eta_{t+2} = 0.1 (0.96)^2 =  0.092$
    - Epoch n: $ \eta_{t+n} = 0.1 (0.96)^n$

In [13]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import StepLR

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# step_size: at how many multiples of epoch you decay
# step_size = 1, after every 2 epoch, new_lr = lr*gamma 
# step_size = 2, after every 2 epoch, new_lr = lr*gamma 

# gamma = decaying factor
scheduler = StepLR(optimizer, step_size=2, gamma=0.96)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    # Decay Learning Rate
    scheduler.step()
    # Print Learning Rate
    print('Epoch:', epoch,'LR:', scheduler.get_lr())
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

Epoch: 0 LR: [0.1]
Iteration: 500. Loss: 0.25232651829719543. Accuracy: 95.66
Epoch: 1 LR: [0.1]
Iteration: 1000. Loss: 0.07672126591205597. Accuracy: 96.47
Epoch: 2 LR: [0.096]
Iteration: 1500. Loss: 0.10065296292304993. Accuracy: 97.07
Epoch: 3 LR: [0.096]
Iteration: 2000. Loss: 0.05262482166290283. Accuracy: 97.62
Epoch: 4 LR: [0.09216]
Iteration: 2500. Loss: 0.021723033860325813. Accuracy: 97.54
Iteration: 3000. Loss: 0.0373525433242321. Accuracy: 97.71


## Pointers on Step-wise Decay
- You would want to decay your LR gradually when you're training more epochs
    - Converge too fast, to a crappy loss/accuracy, if you decay rapidly
- To decay slower
    - Larger $\gamma$
    - Larger interval of decay

## 2. Reduce on Loss Plateau Decay, Patience=1, Factor=0.1
- Reduce learning rate whenever loss plateaus
    - Patience: number of epochs with no improvement after which learning rate will be reduced
        - Patience = 1 
    - Factor: multiplier to decrease learning rate, lr = lr*factor $\rightarrow \gamma$
        - Factor=0.1
- Optimization Algorithm 4: SGD Nesterov
    - Modification of SGD Momentum 
        - $v_t = \gamma v_{t-1} + \eta \cdot  \nabla J(\theta - \gamma v_{t-1}, x^{i: i+n}, y^{i:i+n})$
        - $\theta = \theta - v_t$

In [34]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import ReduceLROnPlateau

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# lr = lr * factor 
# mode='max': look for the maximum validation accuracy to track
# patience: number of epochs - 1 where loss plateaus before decreasing LR
        # patience = 0, after 1 bad epoch, reduce LR
# factor = decaying factor
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0, verbose=True)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            # print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))
        
    # Decay Learning Rate, pass validation accuracy for tracking at every epoch
    print('Epoch {} completed'.format(epoch))
    print('Loss: {}. Accuracy: {}'.format(loss.data[0], accuracy))
    print('-'*20)
    scheduler.step(accuracy)

Epoch 0 completed
Loss: 0.06099593639373779. Accuracy: 95.66
--------------------
Epoch 1 completed
Loss: 0.17056599259376526. Accuracy: 96.47
--------------------
Epoch 2 completed
Loss: 0.08728326857089996. Accuracy: 96.82
--------------------
Epoch 3 completed
Loss: 0.056053031235933304. Accuracy: 97.56
--------------------
Epoch 4 completed
Loss: 0.050669003278017044. Accuracy: 97.58
--------------------
Epoch 5 completed
Loss: 0.04032094031572342. Accuracy: 97.45
--------------------
Epoch     5: reducing learning rate of group 0 to 1.0000e-02.
Epoch 6 completed
Loss: 0.0028860687743872404. Accuracy: 97.97
--------------------
Epoch 7 completed
Loss: 0.0026359560433775187. Accuracy: 98.01
--------------------
Epoch 8 completed
Loss: 0.0034447305370122194. Accuracy: 98.04
--------------------
Epoch 9 completed
Loss: 0.026919852942228317. Accuracy: 97.98
--------------------
Epoch     9: reducing learning rate of group 0 to 1.0000e-03.


## 2. Reduce on Loss Plateau Decay, Patience=1, Factor=0.5
- Reduce learning rate whenever loss plateaus
    - Patience: number of epochs with no improvement after which learning rate will be reduced
        - Patience = 1 
    - Factor: multiplier to decrease learning rate, lr = lr*factor
        - Factor=0.5
- Optimization Algorithm 4: SGD Nesterov
    - Modification of SGD Momentum 
        - $v_t = \gamma v_{t-1} + \eta \cdot  \nabla J(\theta - \gamma v_{t-1}, x^{i: i+n}, y^{i:i+n})$
        - $\theta = \theta - v_t$

In [35]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Set seed
torch.manual_seed(0)

# Where to add a new import
from torch.optim.lr_scheduler import ReduceLROnPlateau

'''
STEP 1: LOADING DATASET
'''

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

'''
STEP 7: INSTANTIATE STEP LEARNING SCHEDULER CLASS
'''
# lr = lr * factor 
# mode='max': look for the maximum validation accuracy to track
# patience: number of epochs - 1 where loss plateaus before decreasing LR
        # patience = 0, after 1 bad epoch, reduce LR
# factor = decaying factor
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=0, verbose=True)

'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images to a Torch Variable
                images = Variable(images.view(-1, 28*28))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            # print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))
        
    # Decay Learning Rate, pass validation accuracy for tracking at every epoch
    print('Epoch {} completed'.format(epoch))
    print('Loss: {}. Accuracy: {}'.format(loss.data[0], accuracy))
    print('-'*20)
    scheduler.step(accuracy)

Epoch 0 completed
Loss: 0.06099593639373779. Accuracy: 95.66
--------------------
Epoch 1 completed
Loss: 0.17056599259376526. Accuracy: 96.47
--------------------
Epoch 2 completed
Loss: 0.08728326857089996. Accuracy: 96.82
--------------------
Epoch 3 completed
Loss: 0.056053031235933304. Accuracy: 97.56
--------------------
Epoch 4 completed
Loss: 0.050669003278017044. Accuracy: 97.58
--------------------
Epoch 5 completed
Loss: 0.04032094031572342. Accuracy: 97.45
--------------------
Epoch     5: reducing learning rate of group 0 to 5.0000e-02.
Epoch 6 completed
Loss: 0.0020523075945675373. Accuracy: 98.04
--------------------
Epoch 7 completed
Loss: 0.0019354888936504722. Accuracy: 97.92
--------------------
Epoch     7: reducing learning rate of group 0 to 2.5000e-02.
Epoch 8 completed
Loss: 0.003255807561799884. Accuracy: 98.02
--------------------
Epoch     8: reducing learning rate of group 0 to 1.2500e-02.
Epoch 9 completed
Loss: 0.019222687929868698. Accuracy: 98.05
-------

## Pointers on Reduce on Loss Pleateau Decay
- In these examples, we used patience=1 because we are running few epochs
    - You should look at a larger patience such as 5 if for example you ran 500 epochs. 
- You should experiment with 2 properties 
    - Patience
    - Decay factor 

## Summary
- Learning Rate Intuition
    - Update parameters so model can churn output closer to labels
    - Gradual parameter updates
- Learning Rate Pointers
    - If we set $\eta$ to be a **large value** $\rightarrow$ learn too much (rapid learning)
    - If we set $\eta$ to be a **small value** $\rightarrow$ learn too little (slow learning)
- Learning Rate Schedules
    - Step-wise Decay
    - Reduce on Loss Plateau Decay
- Step-wise Decay
    - Every 1 epoch
    - Every 2 epoch
    - Every 1 epoch, larger gamma
- Step-wise Decay Pointers
    - Decay LR gradually
        - Larger $\gamma$
        - Larger interval of decay (increase epoch)
- Reduce on Loss Plateau Decay
    - Patience=1, Factor=1
    - Patience =1, Factor=0.5
- Pointers on Reduce on Loss Plateau Decay
    - Larger patience with more epochs
    - 2 properties to experiment
        - Patience
        - Decay factor

